In [4]:
import cv2
import numpy as np
import mediapipe as mp
import time
import math

# Initialize MediaPipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7, min_tracking_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

# Keyboard layout (lowercase will be handled by caps/shift toggle)
keys = [
    ["1","2","3","4","5","6","7","8","9","0"],  # Numbers row
    ["Q","W","E","R","T","Y","U","I","O","P"],
    ["A","S","D","F","G","H","J","K","L"],
    ["Z","X","C","V","B","N","M"],
    ["SHIFT","SPACE","ENTER","BACK","CLEAR"]  # Special row
]

keyboard_pos = {}
typed_text = ""
selected_key = None
click_delay = 0.5
last_click_time = 0
caps = True  # Start with uppercase enabled

# Draw the keyboard on the screen
def draw_keyboard(img):
    global keyboard_pos
    keyboard_pos = {}

    h, w, _ = img.shape
    key_size = 70
    spacing = 12

    # Special key sizes
    special_sizes = {
        "BACK": 100,
        "CLEAR": 100,
        "SHIFT": 100,
        "ENTER": 120,
        "SPACE": 500  # stretched spacebar
    }

    total_rows = len(keys)
    keyboard_height = total_rows * (key_size + spacing)
    start_y = h - keyboard_height - 40  # bottom padding

    for i, row in enumerate(keys):
        # Row width calculation
        row_width = sum([special_sizes.get(k, key_size) + spacing for k in row]) - spacing
        start_x = (w - row_width) // 2  # Center row

        x = start_x
        y = start_y + i * (key_size + spacing)

        for key in row:
            k_size = special_sizes.get(key, key_size)
            keyboard_pos[key] = (x, y, x + k_size, y + key_size)

            overlay = img.copy()
            if key in ["BACK","SPACE","CLEAR","SHIFT","ENTER"]:
                cv2.rectangle(overlay, (x, y), (x + k_size, y + key_size), (100, 0, 200), -1)
            else:
                cv2.rectangle(overlay, (x, y), (x + k_size, y + key_size), (0, 100, 255), -1)

            alpha = 0.4
            cv2.addWeighted(overlay, alpha, img, 1 - alpha, 0, img)
            cv2.rectangle(img, (x, y), (x + k_size, y + key_size), (255, 255, 255), 2, cv2.LINE_AA)

            # Adjust labels for caps/shift
            display_key = key
            if len(key) == 1:  # alphabets/numbers
                display_key = key.upper() if caps else key.lower()

            font_scale = 1 if key not in ["SPACE","BACK","CLEAR","SHIFT","ENTER"] else 0.7
            (text_w, text_h), _ = cv2.getTextSize(display_key, cv2.FONT_HERSHEY_SIMPLEX, font_scale, 2)
            text_x = x + (k_size - text_w) // 2
            text_y = y + (key_size + text_h) // 2
            cv2.putText(img, display_key, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, font_scale,
                        (255, 255, 255), 2, cv2.LINE_AA)

            x += k_size + spacing

    return img

# Detect pinch between index and thumb
def is_pinching(index_tip, thumb_tip, threshold=28):
    dist = math.hypot(index_tip[0] - thumb_tip[0], index_tip[1] - thumb_tip[1])
    return dist < threshold

# Check which key is being pinched
def check_key_pressed(finger_pos):
    global selected_key, typed_text, last_click_time, caps

    for key, (x1, y1, x2, y2) in keyboard_pos.items():
        fx, fy = finger_pos
        if x1 < fx < x2 and y1 < fy < y2:
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 150), -1)
            cv2.addWeighted(frame, 0.6, frame, 0.4, 0, frame)
            cv2.putText(frame, key, (x1 + 15, y1 + 45), cv2.FONT_HERSHEY_SIMPLEX, 1,
                        (0, 0, 0), 3, cv2.LINE_AA)

            if time.time() - last_click_time > click_delay:
                if key == "BACK":
                    typed_text = typed_text[:-1]
                elif key == "SPACE":
                    typed_text += " "
                elif key == "CLEAR":
                    typed_text = ""
                elif key == "SHIFT":
                    caps = not caps  # toggle caps/shift
                elif key == "ENTER":
                    typed_text += "\n"
                else:
                    typed_text += key.upper() if caps else key.lower()
                last_click_time = time.time()
            break

# Open webcam
cap = cv2.VideoCapture(0)
cap.set(3, 1280)
cap.set(4, 720)

while True:
    success, frame = cap.read()
    if not success:
        break
    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    frame = draw_keyboard(frame)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            lm_list = []
            h, w, _ = frame.shape
            for lm in hand_landmarks.landmark:
                cx, cy = int(lm.x * w), int(lm.y * h)
                lm_list.append((cx, cy))

            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            if lm_list:
                index_tip = lm_list[8]
                thumb_tip = lm_list[4]

                if is_pinching(index_tip, thumb_tip, threshold=35):
                    check_key_pressed(index_tip)

    # ===== Centered Typed Text Box =====
    h, w, _ = frame.shape
    box_w, box_h = 1000, 100
    x1 = (w - box_w) // 2
    y1 = 40
    x2 = x1 + box_w
    y2 = y1 + box_h

    overlay = frame.copy()
    cv2.rectangle(overlay, (x1, y1), (x2, y2), (30, 30, 30), -1)
    alpha = 0.5
    cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)

    roi = frame[y1:y2, x1:x2]
    blur = cv2.GaussianBlur(roi, (25, 25), 30)
    frame[y1:y2, x1:x2] = cv2.addWeighted(blur, 0.6, roi, 0.4, 0)

    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 150), 2, cv2.LINE_AA)

    # Display typed text with multiline support
    y_offset = y2 - 20
    for line in typed_text.split("\n")[-3:]:  # show last 3 lines max
        cv2.putText(frame, line, (x1 + 15, y_offset),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 3, cv2.LINE_AA)
        y_offset += 60

    cv2.imshow("HoloType", frame)

    if cv2.waitKey(1) & 0xFF == 27:  # ESC to exit
        break

cap.release()
cv2.destroyAllWindows()
